# Airport On-Time Data
- The data comes the [Bureau of Transport Statistics](https://www.transtats.bts.gov/)
- For now I'm only using data for 2018, but historical data exists since 1987.

In [11]:
# import libraries
import pandas as pd
import glob

year = '2019'
data_path = 'C:/Users/Zaca/Documents/Datasets/flights/' + year + '/'

## Column Description

In [12]:
cols = pd.read_csv('C:/Users/Zaca/Documents/Datasets/flights/columns.txt', sep=',', names=['colname', 'coldesc'], index_col= False)
cols

,colname,coldesc
0,FlightDate,Flight Date (yyyymmdd)
1,Month,Month
2,DayofMonth,Day of Month
3,DayOfWeek,Day of Week
4,Reporting_Airline,Unique Carrier Code
5,Flight_Number_Reporting_Airline,Flight Number
6,OriginAirportID,Airport ID
7,Origin,Origin Airport
8,OriginCityName,City Name
9,OriginState,Origin Airport State Code


In [13]:
# load data
test_flights = pd.read_csv(data_path + '01.csv', usecols=cols['colname'])

# Data Cleaning

In [14]:
# check dtypes
test_flights.dtypes

Month                                int64
DayofMonth                           int64
DayOfWeek                            int64
FlightDate                          object
Reporting_Airline                   object
Flight_Number_Reporting_Airline      int64
OriginAirportID                      int64
Origin                              object
OriginCityName                      object
OriginState                         object
DestAirportID                        int64
Dest                                object
DestCityName                        object
DestState                           object
CRSDepTime                           int64
DepTime                            float64
DepDelay                           float64
TaxiOut                            float64
WheelsOff                          float64
WheelsOn                           float64
TaxiIn                             float64
CRSArrTime                           int64
ArrTime                            float64
ArrDelay   

*It seems that all columns are in the correct data format*

# Checking for NaNs

In [15]:
# get proportion of nan values in each col.
test_flights.isna().sum()/test_flights.shape[0]

Month                              0.000000
DayofMonth                         0.000000
DayOfWeek                          0.000000
FlightDate                         0.000000
Reporting_Airline                  0.000000
Flight_Number_Reporting_Airline    0.000000
OriginAirportID                    0.000000
Origin                             0.000000
OriginCityName                     0.000000
OriginState                        0.000000
DestAirportID                      0.000000
Dest                               0.000000
DestCityName                       0.000000
DestState                          0.000000
CRSDepTime                         0.000000
DepTime                            0.028001
DepDelay                           0.028006
TaxiOut                            0.028453
WheelsOff                          0.028453
WheelsOn                           0.029215
TaxiIn                             0.029215
CRSArrTime                         0.000000
ArrTime                         

### About NaNs:
* Most NaNs seem to happen in the columns that describe the reason for the delay. I am still very interested in the data contained in these columns, so I was really hoping not to drop them.

In [16]:
# check the proportion of nans in flights with actual delays
# because some flights make up for departure delays during airtime, I'm going to focus on arrival delays
print(test_flights[test_flights.ArrDelay > 15].isna().sum())

# interesting, there are no nans when we look at arrival delay which means all of them are explained.
# what about depature delay?
print(test_flights[test_flights.DepDelay > 15].isna().sum())

# when we only consider departure delays now we see nans. I'm not really going to do the math but
# this suggests that perhaps departure delays does not always mean an arrival delay and in that case there are
# no values on the delay cause.

Month                              0
DayofMonth                         0
DayOfWeek                          0
FlightDate                         0
Reporting_Airline                  0
Flight_Number_Reporting_Airline    0
OriginAirportID                    0
Origin                             0
OriginCityName                     0
OriginState                        0
DestAirportID                      0
Dest                               0
DestCityName                       0
DestState                          0
CRSDepTime                         0
DepTime                            0
DepDelay                           0
TaxiOut                            0
WheelsOff                          0
WheelsOn                           0
TaxiIn                             0
CRSArrTime                         0
ArrTime                            0
ArrDelay                           0
Cancelled                          0
Diverted                           0
CRSElapsedTime                     0
A

* It seems that these cols only have values when the flight was actually delayed more than 15 min on arrival, if that is the case, then it makes sense to replace these nans by zero.

In [17]:
# making a list of the delay reason cols and filling with 0
delay_cols = ['CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']
test_flights[delay_cols] = test_flights[delay_cols].fillna(value=0)

In [18]:
print(test_flights.isna().sum())

Month                                  0
DayofMonth                             0
DayOfWeek                              0
FlightDate                             0
Reporting_Airline                      0
Flight_Number_Reporting_Airline        0
OriginAirportID                        0
Origin                                 0
OriginCityName                         0
OriginState                            0
DestAirportID                          0
Dest                                   0
DestCityName                           0
DestState                              0
CRSDepTime                             0
DepTime                            16352
DepDelay                           16355
TaxiOut                            16616
WheelsOff                          16616
WheelsOn                           17061
TaxiIn                             17061
CRSArrTime                             0
ArrTime                            17061
ArrDelay                           18022
Cancelled       

In [19]:
# the rest of nans only account for about 3% of the values in each column, so I will just drop the rows.
# we have enough data...
print('# Flights before drop: ', test_flights.shape[0])
test_flights.dropna(inplace=True)
print('# Flights after drop: ', test_flights.shape[0])

# Flights before drop:  583985
# Flights after drop:  565963


**Looks like this is pretty much everything for the initial cleaning, I will keep the procedural code for the rationale. However, we need to do this iteratively for every month in the database (computer breaks when I load all at once)**

In [20]:
# just repeating the cleaning process above for all months in 2019
def clean_df(filename):
    
    print(filename)
    df = pd.read_csv(filename, usecols=cols['colname'])   
    delay_cols = ['CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']
    df[delay_cols] = df[delay_cols].fillna(value=0)
    
    before = df.shape[0]
    df.dropna(inplace=True)
    print('# Flights dropped: ', before - df.shape[0])
    
    return df
    

In [21]:
# get all files for 2019
filenames = glob.glob(data_path + '*.csv')

In [22]:
flights = [clean_df(f) for f in filenames]

C:/Users/Zaca/Documents/Datasets/flights/2019\01.csv
# Flights dropped:  18022
C:/Users/Zaca/Documents/Datasets/flights/2019\02.csv
# Flights dropped:  16861
C:/Users/Zaca/Documents/Datasets/flights/2019\03.csv
# Flights dropped:  13569
C:/Users/Zaca/Documents/Datasets/flights/2019\04.csv
# Flights dropped:  16003
C:/Users/Zaca/Documents/Datasets/flights/2019\05.csv
# Flights dropped:  15051
C:/Users/Zaca/Documents/Datasets/flights/2019\06.csv
# Flights dropped:  15804
C:/Users/Zaca/Documents/Datasets/flights/2019\07.csv
# Flights dropped:  15248
C:/Users/Zaca/Documents/Datasets/flights/2019\08.csv
# Flights dropped:  13110
C:/Users/Zaca/Documents/Datasets/flights/2019\09.csv
# Flights dropped:  11263
C:/Users/Zaca/Documents/Datasets/flights/2019\10.csv
# Flights dropped:  6377
C:/Users/Zaca/Documents/Datasets/flights/2019\11.csv
# Flights dropped:  5346
C:/Users/Zaca/Documents/Datasets/flights/2019\12.csv
# Flights dropped:  7151


In [23]:
flights = pd.concat(flights, ignore_index= True)

In [24]:
flights.shape

(7268232, 35)

In [25]:
flights.to_csv('C:/Users/Zaca/Documents/Datasets/flights/' + year + '.csv')